In [ ]:
model_save_path = "saved_model_cnn1"
cols = ["KOSPI", "KOSPI_START", "KOSPI_HIGH", "KOSPI_LOW", "NASDAQ", "DOW", "CR", "GOLD"]
len_x_ARMA = 60
len_y_nextday = 1
scale_method = "minmax"

In [ ]:
from kospi_predict import Crawler

crawler = Crawler(crawl_page_max=30, perPage=100)
# crawler.crawlData(cols, save=True)
crawler.loadFromSavedFile(cols)
df_crawled = crawler.removeNan()
df_crawled = df_crawled.loc[:2000]
df_crawled

In [ ]:
from kospi_predict import DataPreprocessor
from sklearn.preprocessing import MinMaxScaler as MinMaxScaler

dpp = DataPreprocessor(df_crawled, cols, scale_method, model_save_path)
dpp.sortByDate()
dpp.makeDiffByRange(1, len_x_ARMA)
dpp.makeAR(0, len_x_ARMA)
dpp.makeMA(2, len_x_ARMA)
dpp.makeTargetYs(len_y_nextday)
dpp.cutoffData(len_x_ARMA, len_y_nextday)

In [ ]:
# df = dpp.df[500:1500]
# df.reset_index(drop=True, inplace=True)
dpp.df

In [ ]:
import cv2
import numpy as np
import pandas as pd

def rescalingByMinMax(arr):
    result = []
    max_val = max(arr)
    min_val = min(arr)
    for val in arr:
        new_val = (val - min_val) / (max_val - min_val)
        result.append(new_val)
    return result

def makeImage3D(df):
    img_list = []
    label_list = []
    date_list = []
    y_list = []
    for item in df.iterrows():
        this_mat = []
        for col in cols:
            this_col_ar_mat = rescalingByMinMax(df.loc[item[0], [x for x in df.columns.to_list() if "{}_AR".format(col) in x]].to_list()) #AR
            this_col_ma_mat = rescalingByMinMax(df.loc[item[0], [x for x in df.columns.to_list() if "{}_MA".format(col) in x]].to_list()) #MA
            this_col_diff_mat = rescalingByMinMax(df.loc[item[0], [x for x in df.columns.to_list() if "{}_DIFF".format(col) in x]].to_list()) #D
            this_row = []
            for idx, _ in enumerate(this_col_ar_mat):
                this_row.append([this_col_ar_mat[idx], this_col_ma_mat[idx], this_col_diff_mat[idx]])
            this_mat.append(this_row)
        this_mat = np.array(this_mat)
        img_list.append(this_mat)

        #라벨 체크
        this_label = np.nan
        if item[0]+1 < len(df): #0 1 2 ... 29 / 30
            this_label = 0
            if df.loc[item[0]+1, "KOSPI"] > df.loc[item[0], "KOSPI"]: #item[0]+1 : 다음날
                this_label = 1
        label_list.append(this_label)

        #라벨값과 함께 이미지로 저장
        cv2.imwrite("data_cnn/{}_{}.png".format(str(this_label), df.loc[item[0], "date"]), this_mat * 255)

        #날짜 리스트 입력
        date_list.append(df.loc[item[0], "date"])

        #y_list 입력
        this_y_list = []
        for y in [x for x in df.columns.to_list() if "Y_" in x]:
            this_y_list.append(df.loc[item[0], y])
        y_list.append(this_y_list)

    img_list = np.array(img_list[:-1])
    label_list = np.array(label_list[:-1], dtype=np.uint8)
    date_list = np.array(date_list[:-1])
    y_list = np.array(y_list[:-1])
    df_result = pd.DataFrame({"date" : date_list, "label" : label_list})

    return img_list, label_list, date_list, y_list, df_result

img_list, label_list, date_list, y_list, df_ref = makeImage3D(dpp.df)
img_list.shape, label_list.shape, date_list.shape, y_list.shape

In [ ]:
df_ref

In [ ]:
def makeImage2D(df):
    #행 : X_*_DIFF_AR, X_*_DIFF_MA  *  원천 X 갯수(8개라면 8*2 = 16행)
    #열 : ARMA 갯수. 20이라면 20 = 20열
    col_made_list = [
        "X_*_AR", #8행
        "X_*_DIFF_AR", #8행
        "X_*_MA", #8행
        "X_*_DIFF_MA" #8행
    ]
    img_list = []
    label_list = []
    date_list = []
    y_list = []
    for item in df.iterrows():
        this_mat_list = []
        for col in cols: #8개 ["KOSPI", "KOSPI_START", "KOSPI_HIGH", "KOSPI_LOW", "NASDAQ", "DOW", "CR", "GOLD"]
            for col_made in col_made_list: #4개 ["X_*_AR", "X_*_DIFF_AR", "X_*_MA", "X_*_DIFF_MA"]
                this_col = col_made.replace("*", col) #X_KOSPI_AR
                this_row_list = []
                for i in range(0, len_x_ARMA): #20개. 0~20
                    if "MA" in this_col:
                        i += 2
                    this_col_made = "{}{}".format(this_col, i) #X_KOSPI_AR0
                    this_row_list.append([df.loc[item[0], this_col_made]])
                this_mat_list.append(this_row_list)
        this_mat = np.array(this_mat_list)
        this_mat = this_mat
        img_list.append(this_mat)

        #라벨 체크
        this_label = np.nan
        if item[0]+1 < len(df): #0 1 2 ... 29 / 30
            this_label = 0
            if df.loc[item[0]+1, "KOSPI"] > df.loc[item[0], "KOSPI"]: #item[0]+1 : 다음날
                this_label = 1
        label_list.append(this_label)

        #라벨값과 함께 이미지로 저장
        cv2.imwrite("data_cnn/{}_{}.png".format(str(this_label), df.loc[item[0], "date"]), this_mat * 255)

        #날짜 리스트 입력
        date_list.append(df.loc[item[0], "date"])

        #y_list 입력
        this_y_list = []
        for y in [x for x in df.columns.to_list() if "Y_" in x]:
            this_y_list.append(df.loc[item[0], y])
        y_list.append(this_y_list)

    img_list = np.array(img_list)
    label_list = np.array(label_list)
    date_list = np.array(date_list)
    y_list = np.array(y_list)
    df_result = pd.DataFrame({"date" : date_list, "label" : label_list})

    return img_list, label_list, date_list, y_list, df_result

# img_list, label_list, date_list, y_list, _ = makeImage2D(dpp.df)
# img_list.shape, label_list.shape, date_list.shape, y_list.shape

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(img_list[:-1], label_list[:-1], test_size=0.2, shuffle=False, random_state=1234)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(img_list[0].shape[0], img_list[0].shape[1], img_list[0].shape[2])),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  # tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(2, activation="softmax")
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()
epochs=200
history = model.fit(
  x_train,
  y_train,
  epochs=epochs,
  batch_size = 10,
  validation_split = 0.1,
  verbose = 1
)

In [ ]:
df_history = pd.DataFrame(history.history)
df_history.plot()

In [ ]:
preds = model.predict(x_test)
test_results = []
for idx, pred in enumerate(preds):
    this_label = y_test[idx]
    this_predict = np.argmax(pred)
    acc = 0
    if this_label == this_predict:
        acc = 1
    test_results.append({"label" : this_label, "predict" : this_predict, "acc" : acc})

test_results = pd.DataFrame(test_results)
test_results

In [ ]:
acc_ratio = test_results["acc"].sum() / len(test_results)
print("테스트 정확도 : {}/{} ({:.2f})".format(test_results["acc"].sum(), len(test_results), acc_ratio))

최근 날짜로 테스트

In [ ]:
crawler_pred = Crawler(crawl_page_max=1, perPage=100) #이전 추세도 볼겸 넉넉히 수집
crawler_pred.crawlData(cols)
df_crawled_pred = crawler_pred.removeNan()
df_crawled_pred

In [ ]:
#아래는 예측을 위한 전처리
dpp_pred = DataPreprocessor(df_crawled_pred.loc[:100], cols, scale_method, model_save_path)
dpp_pred.sortByDate()
dpp_pred.makeDiffRatio()
dpp_pred.scalingForPredict()
dpp_pred.makeAR(0, len_x_ARMA)
dpp_pred.makeMA(2, len_x_ARMA)
dpp_pred.cutoffData(len_x_ARMA, 0)

In [ ]:
dpp_pred.df

In [ ]:
img_pred_list, label_pred_list, date_list, _ = makeImage2D(dpp_pred.df, cut_latest_data=False)
len(img_pred_list), len(label_pred_list), len(date_list)

In [ ]:
label_pred_list

In [ ]:
before_day = 1
img_pred = np.array([img_pred_list[-before_day]])
pred = model.predict(img_pred)
test_results = []
predict = np.argmax(pred)
print("{} predict : {}, real : {}".format(date_list[-before_day], predict, label_pred_list[-before_day]))

# CNN + 예측

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(img_list, y_list, test_size=0.2, shuffle=False, random_state=8699)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(24, 20, 1)),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(20, activation="relu")
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mse',
            optimizer=optimizer,
            metrics=['mae', 'mse'])

model.summary()
epochs=5000
history = model.fit(
  x_train,
  y_train,
  epochs=epochs,
  validation_split = 0.1,
  verbose = 1
)